In [8]:
import re
import os
import json
import shutil
import pandas as pd

In [9]:
def sanitize_filename(name: str) -> str:
    return re.sub(r'[<>:"/\\|?*]', '_', name)

def export_pdfs_from_excel(
        excel_path: str,
        output_folder: str,
        sheet_name: str
):
    df = pd.read_excel(excel_path, sheet_name=sheet_name)

    os.makedirs(output_folder, exist_ok=True)

    for idx, row in df.iterrows():
        file_path = row.get("File Attachments")
        if not isinstance(file_path, str) or not file_path.lower().endswith(".pdf"):
            continue

        file_path = file_path.strip()
        file_path = file_path.replace("file://", "")
        file_path = os.path.expanduser(file_path)

        if not os.path.exists(file_path):
            print(f"Warning: File not found - {file_path}")
            continue

        author = str(row.get("Authors", "")).strip().replace("/", "_")
        year = str(row.get("Year", "")).strip()
        title = str(row.get("Title", "")).strip().replace("/", "_")
        new_filename = f"{author} - {year} - {title}.pdf"
        new_filename = sanitize_filename(new_filename)
        if len(new_filename) > 180:
            new_filename = new_filename[:170] + ".pdf"

        dest_path = os.path.join(output_folder, new_filename)
        shutil.copyfile(file_path, dest_path)

def excel_to_json_with_links(
        excel_path: str,
        output_json_path: str,
        pdf_relative_path: str,
        sheet_name: str
):
    df = pd.read_excel(excel_path, sheet_name=sheet_name)

    if "File Attachments" in df.columns:
        df = df.drop(columns=["File Attachments"])

    records = []
    for _, row in df.iterrows():
        author = sanitize_filename(str(row.get("Authors", "")).strip())
        year = sanitize_filename(str(row.get("Year", "")).strip())
        title_raw = str(row.get("Title", "")).strip()
        title_clean = sanitize_filename(title_raw)

        pdf_filename = f"{author} - {year} - {title_clean}.pdf"
        pdf_link = os.path.join(pdf_relative_path, pdf_filename).replace("\\", "/")
        title_with_link = f'<a href="{pdf_link}" target="_blank">{title_raw}</a>'

        row_dict = row.to_dict()
        row_dict["Title"] = title_with_link
        records.append(row_dict)

    with open(output_json_path, "w", encoding="utf-8") as f:
        json.dump(records, f, indent=2, ensure_ascii=False)

In [7]:
export_pdfs_from_excel(
    excel_path="./bib_others/bib_others.xlsx",
    output_folder="./bib_others/files",
    sheet_name="Sheet1"
)

In [10]:
excel_to_json_with_links(
    excel_path="./bib_others/bib_others.xlsx",
    output_json_path="./bib_others/bib_others.json",
    pdf_relative_path="./bib_others/files",
    sheet_name="Sheet1"
)

In [11]:
excel_to_json_with_links(
    excel_path="./bib_authors/bib_authors.xlsx",
    output_json_path="./bib_authors/bib_authors.json",
    pdf_relative_path="./bib_authors/files",
    sheet_name="Sheet1"
)

In [13]:
excel_to_json_with_links(
    excel_path="./bib_original/bib_original.xlsx",
    output_json_path="./bib_original/bib_original.json",
    pdf_relative_path="./bib_original/files",
    sheet_name="Sheet1"
)